In [ ]:
import pandas as pd
import time
import numpy as np
import json
from nltk import ngrams
from sklearn.feature_extraction import text
from itsdangerous import URLSafeSerializer
import hashlib
import operator
import itertools
from itertools import combinations, chain
import import_ipynb
import EntropyBasedItemsetGenerationModule as entropy

In [ ]:
class PreprocessApriori:
    def __init__(self, masterData, softwareToolGroupDict, columns,list_var):
        self.masterData = masterData
        self.softwareToolGroupDict = softwareToolGroupDict
        self.columns = columns
        self.list_var=list_var
    
    def getSubsetNotNA(self, df, columnName):
        data = df.copy()
        data = data[data[columnName].notna()]
        return data
        
    def getSubsetForApriori(self, df, columnName, columnvalue):
        df.dropna(subset=[columnName], inplace=True)
        data = df.loc[df[columnName] == columnValue]
        return data
        
    def preprocessSoftwareToolsGroup(self, df, groups):
        data = df.copy()
        var = 'SoftwareTools'
        data[var]=data[var].str.split('##')
        for i,v in data.iterrows():
            #print(data['SoftwareTools'][i])
            l=[]
            if(isinstance(v[var],list)):
                for k in v['SoftwareTools']:
                    if(k!=''):
                        t=k.rstrip("\n")
                        h=t.strip()
                        #print(h)
                        l.append(h.lower())
            data['SoftwareTools'][i]=l
            
        for i,v in data.iterrows():
            l=[]
            for k in v['SoftwareTools']:
                if(k in groups['l1']):
                    l.append("Adobe")
                if(k in groups['l2']):
                    l.append("Cloud Computing and Big Data")
                if(k in groups['l3']):
                    l.append("Python tools")
                if(k in groups['l4']):
                    l.append("Microsoft Tools")
                if(k in groups['l5']):
                    l.append("DBMS Tools")
                if(k in groups['l6']):
                    l.append("Machine Learning Tools")
                if(k in groups['l7']):
                    l.append("Computer Network")
            l_str = '##'.join(l)
            data['SoftwareTools'][i]=l_str
        return data
    
    def preprocessTier(self,data):
        data['TierLevel']=data['TierLevel'].replace(to_replace ="1.0",value ="1")
        data['TierLevel']=data['TierLevel'].replace(to_replace ="2.0",value ="2")
        data['TierLevel']=data['TierLevel'].replace(to_replace ="3.0",value ="3")
        return data
    
    def columnwiseItemPreprocess(self,data,columns):
        df=data.copy()
        for column in columns:
            #print(column)
            if(isinstance(df[column][1],str)):
                df[column]=df[column].str.strip('##').str.split('##')
                for i,v in df.iterrows():
                    l=[]
                    #print(v)
                    #print(type(v[column]))
                    if(isinstance(v[column],list)):
                        for k in v[column]:
                            #print(k)
                            if(k!=''):
                                l.append(column+"_"+k)
                    #print(l)
                    df[column][i]=l
        return df
    
    def columnwiseItemGeneration(self,data,columns):
        df=data.copy()
        df['combinedItems_noCourse']=np.empty((len(df), 0)).tolist()
        for i,v in df.iterrows():
            l=df['combinedItems_noCourse'][i]
            for column in columns:
                #print(v)
                #print(type(v[column]))
                if(isinstance(v[column],list)):
                    l=l+v[column]
            #print(l)
            df['combinedItems_noCourse'][i]=l
        return df
    
    def allColumnsItemGeneration(self, df, columnName, columnValue, gradeColumnName, courseColumnName, addCourses = True):
        data_full = df.copy()
        self.generateValidCoursesItem(data_full, columnName, columnValue, gradeColumnName, courseColumnName)
        data_full1 = self.getSubsetNotNA(df, columnName)
        data_full1 = self.preprocessTier(data_full1)
        data_full2 = data_full1.merge(self.validCoursesData.drop_duplicates(subset=['USN']), how ='left')
        data_main = self.getSubsetForApriori(data_full2, columnName, columnValue)
        dataPreProcess = self.preprocessSoftwareToolsGroup(data_main, self.softwareToolGroupDict)
        dataPreProcess1 = self.columnwiseItemPreprocess(dataPreProcess, self.columns)
        dataPreProcess2 = self.columnwiseItemGeneration(dataPreProcess1, self.columns)
        if(addCourses):
            dataPreProcess2['combinedItemset'] =  dataPreProcess2['ValidCourses'] + dataPreProcess2['combinedItems_noCourse']
        else:
            dataPreProcess2['combinedItemset'] =  dataPreProcess2['combinedItems_noCourse']
        dataPreProcess2=dataPreProcess2.rename({'ExternalCertificatesKey,ExternalCertificatesDomain,WorkshopsOrg,WorkshopsDomain,PublicationNLP': 'OffCampusLearning', 'GeneralSkillsNLP,VolunteeringWork,CoCurricularActivitiesNLP,LanguagesNLP': 'NonAcademicSkills'}, axis=1) 
        return dataPreProcess2
    
    def allColumnsItemGenerationBoolColumn(self, df, columnName, columnValue, gradeColumnName, courseColumnName, addCourses = True):
        data_full = df.copy()
        self.generateValidCoursesItem(data_full, columnName, columnValue, gradeColumnName, courseColumnName)
        data_full1 = self.getSubsetNotNA(df, columnName)
        data_full1 = self.preprocessTier(data_full1)
        data_full2 = data_full1.merge(self.validCoursesData.drop_duplicates(subset=['USN']), how ='left')
        data_main = self.getSubsetForApriori(data_full2, columnName, columnValue)
        dataPreProcessBoolColumn=self.getColumnBoolVal(data_main,self.list_var)
        #dataBool1=self.renameBoolColumns(dataPreProcessBoolColumn)
        boolColumns=self.boolGetListOfColumns(self.columns)
        print(boolColumns)
        print("--------------------------------------")
        #print(dataBool1.columns)
        #dataBool2=self.boolColumnItemPreprocess(dataBool1)
        dataPreProcess = self.preprocessSoftwareToolsGroup(dataPreProcessBoolColumn, self.softwareToolGroupDict)
        dataPreProcess1 = self.columnwiseItemPreprocess(dataPreProcess, boolColumns)
        dataPreProcess2 = self.columnwiseItemGeneration(dataPreProcess1, boolColumns)
        if(addCourses):
            dataPreProcess2['combinedItemset'] =  dataPreProcess2['ValidCourses'] + dataPreProcess2['combinedItems_noCourse']
        else:
            dataPreProcess2['combinedItemset'] =  dataPreProcess2['combinedItems_noCourse']
        dataPreProcess2=dataPreProcess2.rename({'ExternalCertificatesKey,ExternalCertificatesDomain,WorkshopsOrg,WorkshopsDomain,PublicationNLP': 'OffCampusLearning', 'GeneralSkillsNLP,VolunteeringWork,CoCurricularActivitiesNLP,LanguagesNLP': 'NonAcademicSkills'}, axis=1) 
        return dataPreProcess2
        
    def generateValidCoursesItem(self, data, columnName, columnValue, gradeColumnName, courseColumnName):
        obj = entropy.EntropyBasedItemset(data)
        self.validCoursesData = obj.getOptimalGradeForAllCourses(data, columnName, columnValue, gradeColumnName, courseColumnName)
        self.optimal10thMarks = obj.getEntropy10th(data, columnName, columnValue)
        self.optimal12thMarks = obj.getEntropy12th(data, columnName, columnValue)
    
    def getOptimal10thMarks(self):
        return self.optimal10thMarks
    
    def getOptimal10thMarks(self):
        return self.optimal12thMarks
    
    def getValidCoursesItemset(self):
        return self.validCoursesData
    
    def addColumnBool(self,df,var):
        t=""
        t=",".join(var)
        if(len(var)>1):
            listofcols = var
            df[t] = ''  
            for column in listofcols:     
                df[t] = df[t] + ' ' + df[column]
            var=t
        else:
            var=var[0]
        t=t+"_present"
        t=self.renameBoolColumns(t)
        df[var] = df[var].str.replace(" ", '', regex=True)
        df1 = df.replace(np.nan, '', regex=True)
        df1[t]=""
        for index, row in df1.iterrows():
            l=[]
            if(len(row[var])>0):
                l.append(t)
                df1[t][index]=l
            else:
                df1[t][index]=l
        return df1
    
    def getColumnBoolVal(self,data,list_var):
        df=data.copy()
        for var in list_var:
            df=self.addColumnBool(df,var)
        return df
    
    def boolColumnItemPreprocess(self,data):
        df = data.copy()
        for column in self.boolColumns:
            print(column)
            if(isinstance(df[column][1],(int,np.integer))):
                print(column)
                for i,v in df.iterrows():
                    l=[]
                    if(v[column]==1):
                        l.append(column)
                    df[column][i]=l
        return df
   
    def boolGetListOfColumns(self, columns):
        self.boolColumns = columns[:]
        if('SoftwareTools' in self.boolColumns):
            self.boolColumns.remove('SoftwareTools')
            self.boolColumns.append('SoftwareTools_present')
        if('ProgLanguagesNLP' in self.boolColumns):
            self.boolColumns.remove('ProgLanguagesNLP')
            self.boolColumns.append('ProgLanguagesNLP_present')
        if('CoCurricularActivitiesNLP' in self.boolColumns):
            if('NonAcademicSkills_present' not in self.boolColumns):
                self.boolColumns.append('NonAcademicSkills_present')
            self.boolColumns.remove('CoCurricularActivitiesNLP')
        if('GeneralSkillsNLP' in self.boolColumns):
            if('NonAcademicSkills_present' not in self.boolColumns):
                self.boolColumns.append('NonAcademicSkills_present')
            self.boolColumns.remove('GeneralSkillsNLP')
        if('LanguagesNLP' in self.boolColumns):
            if('NonAcademicSkills_present' not in self.boolColumns):
                self.boolColumns.append('NonAcademicSkills_present')
            self.boolColumns.remove('LanguagesNLP')
        if('VolunteeringWork' in self.boolColumns):
            if('NonAcademicSkills_present' not in self.boolColumns):
                self.boolColumns.append('NonAcademicSkills_present')
            self.boolColumns.remove('VolunteeringWork')
        if('ExternalCertificatesKey' in self.boolColumns):
            if('OffCampusLearning_present' not in self.boolColumns):
                self.boolColumns.append('OffCampusLearning_present')
            self.boolColumns.remove('ExternalCertificatesKey')
        if('ExternalCertificatesDomain' in self.boolColumns):
            if('OffCampusLearning_present' not in self.boolColumns):
                self.boolColumns.append('OffCampusLearning_present')
            self.boolColumns.remove('ExternalCertificatesDomain')
        if('WorkshopsDomain' in self.boolColumns):
            if('OffCampusLearning_present' not in self.boolColumns):
                self.boolColumns.append('OffCampusLearning_present')
            self.boolColumns.remove('WorkshopsDomain')
        if('WorkshopsOrg' in self.boolColumns):
            if('OffCampusLearning_present' not in self.boolColumns):
                self.boolColumns.append('OffCampusLearning_present')
            self.boolColumns.remove('WorkshopsOrg')
        if('PublicationNLP' in self.boolColumns):
            if('OffCampusLearning_present' not in self.boolColumns):
                self.boolColumns.append('OffCampusLearning_present')
            self.boolColumns.remove('PublicationNLP')
        return self.boolColumns
   
    def renameBoolColumns(self, colName):
        if(colName=='ExternalCertificatesKey,ExternalCertificatesDomain,WorkshopsOrg,WorkshopsDomain,PublicationNLP_present'):
            return 'OffCampusLearning_present'
        elif(colName=='GeneralSkillsNLP,VolunteeringWork,CoCurricularActivitiesNLP,LanguagesNLP_present'):
            return 'NonAcademicSkills_present'
        else:
            return colName
        
        
    
        

In [726]:
#MODULAR VERSION: 
class Apriori:
    def __init__(self, data, alpha, ls):
        self.data = data
        self.alpha = alpha
        self.ls = ls
        
    def findsubsets(self, uniqueItems, listFormattedPreviousRules, n):
        #return list(map(set, itertools.combinations(s, n)))
        #find combination the first time for all unique items 
        if(len(listFormattedPreviousRules)==0):
            return list(itertools.combinations(uniqueItems, n))
        else:
            #print("In combo")
            return self.getValidCombinations(uniqueItems, listFormattedPreviousRules, n)
        
    def getValidCombinations(self, uniqueItems, listFormattedPreviousRules, n):
        #print("\n\n\nin get valid combis\n\n\n")
        validCombinations = []
        for item in uniqueItems:
            for rule in listFormattedPreviousRules:
                if(item not in rule):
                    if(self.isCombinationValid(item, rule, listFormattedPreviousRules)):
                        temp =[]
                        temp.append(item)
                        l = rule + temp
                        l.sort()
                        validCombinations.append(tuple(l))
        return list(set(validCombinations))
          
    def isCombinationValid(self, item, rule, listFormattedPreviousRules):
        flag = True
        temp = []
        for prevRule in listFormattedPreviousRules:
            prevRule.sort()
            temp.append(prevRule)
        for i in rule:
            y = rule[:]  # fastest way to copy
            y.remove(i)
            y.append(item)
            y.sort()
            if(y not in temp):
                flag = False
                break
        return flag
            
        
    def findItemCount(self, columnName, data):
        item_count={}
        for v,k in data[columnName].iteritems():
            for i in k:
                if i in item_count:
                    item_count[i]=item_count[i]+1
                else:
                    item_count[i]=1
        print("itemcount\n", item_count)
        print("unique items:", len(item_count))
        return item_count
    
    def findLambda(self, item_count):
        lamb = 0
        for val in item_count.values(): 
            lamb += val 
        lamb = lamb / len(item_count) 
        return lamb
    
    def mis(self, item_count):
        sd=self.findLambda(item_count)*(1-self.alpha)
        mis=[]
        misdict={}
        mis_val={}
        for k, v in item_count.items():
            m=item_count[k]-sd
            if(m<self.ls):
                mis.append(self.ls)
                misdict[item_count[k]]=self.ls
                mis_val[k]=self.ls
            else:
                mis.append(m)
                misdict[item_count[k]]=m
                mis_val[k]=m
        return mis_val
    
    def getMinMIS(self, mis_val):
        return min(mis_val.values())
    
    def getIndividualItems(self, mis_val, item_count):
        L_2={}
        for k, v in item_count.items():
            if(item_count[k]>=mis_val[k]):
                L_2[k]=mis_val[k]
        uniqueItems = list(L_2.keys())
        temp= sorted(L_2.items(), key=operator.itemgetter(0))
        L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)
        print("individual items size: ", len(L_21))
        #print("items:", L_21)
        print("L21 Keys: ",uniqueItems)
        return uniqueItems, L_21
    
    def getNItemsetRules(self, n, individualUniqueItems, listFormattedPreviousRules, mis_val, columnName, data, courseNameColumn):
        start_time = time.time()
        CN=self.findsubsets(individualUniqueItems, listFormattedPreviousRules, n)
        if(n>2):
            print("combination:", CN)
        print("time taken to get all combinations: ", time.time() - start_time)
        print("combination size", len(CN))
        print("combinations:",CN[:3])
        #count for each candidate itemset
        start_time = time.time()
        count = self.getSupportOfEachCombination(CN, data, columnName)
        print("time taken to get support of each combination : ", time.time() - start_time)
        #checking criteria for count
        start_time = time.time()
        itemSet={}
        indivUniqueItems =[]
        for k, v in count.items():
                m=count[k][0]*100/data.shape[0]
                if(m > self.getMinMIS(mis_val)):
                    itemSet[k]=count[k]
        
        indivUniqueItems, listFormattedRules = self.getUnqiueValidItemsAndListFormatteRules(list(itemSet.keys()))
        temp= sorted(itemSet.items(), key=operator.itemgetter(0))
        sortedRules=sorted(temp, key=operator.itemgetter(1), reverse=True)
        print("time taken to get valid combinations : ", time.time() - start_time)
        rules={}
        print(type(sortedRules))
        for i in sortedRules:
            print(i[0])
            print(i[1][0])
            print("--------------------------------------------------------------------------------------------------")
            rules[i[0]]=i[1][0]
        print(rules)
        with open(str(n)+'_Rules.json', 'w') as fp:
            fp.write(json.dumps(rules,indent=2))
        if(len(sortedRules)>4):
            print("N:", n,"No of rules:", len(sortedRules),"\nRules: ", sortedRules[:3], type(sortedRules[:3][0]))
        file=open(str(n)+'_'+str(self.alpha)+'_'+str(self.ls)+'_'+courseNameColumn+'_RulesElectiveCourse.txt','w+')
        for items in sortedRules:
            file.writelines([str(items)])
            file.write("\n")
        file.close()
        return indivUniqueItems, listFormattedRules, sortedRules
    
    def getUnqiueValidItemsAndListFormatteRules(self, listOfItemSets):
        #keys will be strings separated by ,
        allItems = []
        listFormattedRules = []
        for itemSet in listOfItemSets:
            l = itemSet.split(',')
            allItems+= l
            listFormattedRules.append(l)
        return list(set(allItems)),listFormattedRules
    
    def getSupportOfEachCombination(self, CN, data, columnName):
        count={}
        for i in CN:
            for index, row in data.iterrows():
                if(set(i).issubset( row[columnName])):
                    strSet = ','.join(set(i))
                    if(strSet in count):
                        count[strSet].append(index)
                    else:
                        count[strSet] = [index]
        new_count={}
        for i in count:
            temp=list(count[i])
            new_count[i]=[len(count[i]), temp]
        return new_count
            
    def generateAllRules(self, L1List, mis_val, columnName, data, start, end, courseNameColumn):
        rules = {}
        indivUniqueItems = L1List
        listFormattedRules = []
        for i in range(start,end+1):
            indivUniqueItems, listFormattedRules, rule = self.getNItemsetRules(i, indivUniqueItems, listFormattedRules, mis_val, columnName, data, courseNameColumn)
            print("for ", i, " number of rules:", len(rule))
            print("indivudal unique items left:", indivUniqueItems)
            print("number of unique items:", len(indivUniqueItems)) 
            print("rules in list format: ", listFormattedRules)
            if(len(rule)==0):
                break
            rules[i] = rule
        return rules
    
    def runApriori(self, columnName, df, start, end, courseNameColumn):
        data = df.copy()
        item_count = self.findItemCount(columnName, data)
        mis_val = self.mis(item_count)
        uniqueItems , L1Set = self.getIndividualItems(mis_val, item_count)
        print("L1 set: ", L1Set)
        rules = self.generateAllRules(uniqueItems, mis_val, columnName, data,start, end, courseNameColumn)
        print("total rules:", len(rules))
        return rules
        
                

In [ ]:
def readJson(filename):
    data = {}
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

In [ ]:
#Parameter
#1: dataset
#2: Tier, Company name, etc
#3: LS
#4: Alpha
#5: list of column names
start_time = time.time()
softwareToolGroupDict = readJson('softwareToolGroups.json')
columns=['Awards',
       'ExternalCertificatesDomain', 'ExternalCertificatesKey',
       'GeneralSkillsNLP', 'Internship Company',
       'Project Domain in Internship', 'LanguagesNLP',
       'College Project Domains',
       'PublicationNLP', 'Domain of Research Work',
       'Scholarships', 'VolunteeringWork',
       'WorkshopsDomain', 'WorkshopsOrg', 'SoftwareTools', 'ProgLanguagesNLP','CoCurricularActivitiesNLP'
]
columnName = 'TierLevel'
columnValue = '1'
boolAprioriVersion=True
boolCourses=True
list_var=[['SoftwareTools'],['ProgLanguagesNLP'],['GeneralSkillsNLP','VolunteeringWork','CoCurricularActivitiesNLP','LanguagesNLP'],['ExternalCertificatesKey','ExternalCertificatesDomain','WorkshopsOrg','WorkshopsDomain','PublicationNLP']]
data_full= pd.read_csv(r"FinalCombined_April.csv")
data_full=data_full.rename({'AwardsNLP': 'Awards', 
            'CompanyNameNLP': 'Company Name',
           'InternshipCompany':'Internship Company',
           'InternshipProjectDomain':'Project Domain in Internship',
           'ProjectDetailDomain':'College Project Domains',
            'ResearchDomain':'Domain of Research Work',
            'ScholarshipsNLP':'Scholarships',
            'ProgLanguagesNLP_present':'ProgrammingLanguages_present'
           }, axis=1)
obj1 = PreprocessApriori(data_full, softwareToolGroupDict, columns,list_var)
if(boolAprioriVersion):
    a=obj1.allColumnsItemGenerationBoolColumn(data_full, columnName, columnValue,'ElectiveCourseGrade', 'ElectiveCourseName', boolCourses)
else:
    a= obj1.allColumnsItemGeneration(data_full, columnName, columnValue,'ElectiveCourseGrade', 'ElectiveCourseName', boolCourses )
a.reset_index(inplace = True)
a=a.drop(['index','Unnamed: 0','Unnamed: 0.1'], axis = 1)


In [727]:
objApriori = Apriori(a, 0.1, 10)
rules = objApriori.runApriori('combinedItemset',a, 2,10, 'ElectiveCourseName')
print("time taken overall ", time.time() - start_time)

itemcount
 {'ElectiveCourseName_advanced algorithms::A and higher': 86, 'Awards_iayp': 1, 'Awards_e-yantra': 1, 'Project Domain in Internship_Image Processing': 220, 'Project Domain in Internship_Algorithms': 29, 'College Project Domains_Cloud computing': 221, 'College Project Domains_Machine Learning': 119, 'College Project Domains_Web Development': 108, 'College Project Domains_Compiler Design': 92, 'College Project Domains_Big Data': 51, 'College Project Domains_Image Processing': 187, 'College Project Domains_Data Structures': 112, 'College Project Domains_Deep Learning': 81, 'College Project Domains_Artificial Intelligence': 28, 'College Project Domains_Micro Processors and Controllers': 16, 'Domain of Research Work_Machine Learning': 7, 'Domain of Research Work_Image Processing': 15, 'Domain of Research Work_Deep Learning': 2, 'Scholarships_cnr': 87, 'NonAcademicSkills_present': 209, 'OffCampusLearning_present': 80, 'ElectiveCourseName_big data::A and higher': 165, 'ElectiveCours

time taken to get support of each combination :  91.39441299438477
time taken to get valid combinations :  0.018904447555541992
<class 'list'>
College Project Domains_Cloud computing,Project Domain in Internship_Image Processing
162
--------------------------------------------------------------------------------------------------
College Project Domains_Cloud computing,NonAcademicSkills_present
156
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,College Project Domains_Cloud computing
155
--------------------------------------------------------------------------------------------------
Project Domain in Internship_Image Processing,NonAcademicSkills_present
153
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,Project Domain in Internship_Image Processing
138
-------------------------------------------------

combination: [('College Project Domains_Algorithms', 'College Project Domains_Operating System', 'ElectiveCourseName_big data::A and higher'), ('College Project Domains_Algorithms', 'ElectiveCourseName_advanced algorithms::A and higher', 'Project Domain in Internship_Image Processing'), ('ElectiveCourseName_big data::A and higher', 'NonAcademicSkills_present', 'ProgLanguagesNLP_present'), ('College Project Domains_IoT', 'NonAcademicSkills_present', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Algorithms', 'College Project Domains_Database Management System', 'OffCampusLearning_present'), ('NonAcademicSkills_present', 'Project Domain in Internship_Image Processing', 'Scholarships_cnr'), ('College Project Domains_Algorithms', 'College Project Domains_Compiler Design', 'College Project Domains_Machine Learning'), ('College Project Domains_Cloud computing', 'ElectiveCourseName_advanced computer networks::A and higher', 'ElectiveCourseName_big data::A and high

time taken to get support of each combination :  27.480741024017334
time taken to get valid combinations :  0.007030010223388672
<class 'list'>
College Project Domains_Image Processing,College Project Domains_Cloud computing,Project Domain in Internship_Image Processing
118
--------------------------------------------------------------------------------------------------
College Project Domains_Cloud computing,Project Domain in Internship_Image Processing,NonAcademicSkills_present
114
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,College Project Domains_Cloud computing,NonAcademicSkills_present
109
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,College Project Domains_Cloud computing,ProgLanguagesNLP_present
97
--------------------------------------------------------------------------------------------

combination: [('College Project Domains_Image Processing', 'ProgLanguagesNLP_present', 'Project Domain in Internship_Cloud computing', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'College Project Domains_Data Structures', 'College Project Domains_Image Processing', 'College Project Domains_Machine Learning'), ('College Project Domains_Image Processing', 'College Project Domains_Operating System', 'ElectiveCourseName_big data::A and higher', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'ElectiveCourseName_big data::A and higher', 'Project Domain in Internship_Image Processing', 'Scholarships_cnr'), ('College Project Domains_Web Development', 'NonAcademicSkills_present', 'ProgLanguagesNLP_present', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'College Project Domains_Database Management System', 'NonAcademicSkills_present', 'Project Domain in

time taken to get support of each combination :  21.214072227478027
time taken to get valid combinations :  0.006947040557861328
<class 'list'>
College Project Domains_Image Processing,College Project Domains_Cloud computing,Project Domain in Internship_Image Processing,NonAcademicSkills_present
84
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,College Project Domains_Cloud computing,ProgLanguagesNLP_present,Project Domain in Internship_Image Processing
73
--------------------------------------------------------------------------------------------------
College Project Domains_Database Management System,College Project Domains_Cloud computing,College Project Domains_Image Processing,Project Domain in Internship_Image Processing
69
--------------------------------------------------------------------------------------------------
College Project Domains_Image Processing,College Project Domains_Cl

combination: [('College Project Domains_Cloud computing', 'College Project Domains_Data Structures', 'College Project Domains_Database Management System', 'NonAcademicSkills_present', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'College Project Domains_Data Structures', 'College Project Domains_Image Processing', 'NonAcademicSkills_present', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'College Project Domains_Image Processing', 'ProgLanguagesNLP_present', 'Project Domain in Internship_Cloud computing', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Cloud computing', 'College Project Domains_Operating System', 'NonAcademicSkills_present', 'ProgLanguagesNLP_present', 'Project Domain in Internship_Image Processing'), ('College Project Domains_Algorithms', 'College Project Domains_Cloud computing', 'College Project Domains_Machine Learning', 'NonAcademicSkills_pr

time taken to get support of each combination :  6.826480388641357
time taken to get valid combinations :  0.0019953250885009766
<class 'list'>
NonAcademicSkills_present,College Project Domains_Database Management System,Project Domain in Internship_Image Processing,College Project Domains_Image Processing,College Project Domains_Cloud computing
49
--------------------------------------------------------------------------------------------------
NonAcademicSkills_present,Project Domain in Internship_Image Processing,College Project Domains_Image Processing,College Project Domains_Cloud computing,ElectiveCourseName_big data::A and higher
47
--------------------------------------------------------------------------------------------------
ProgLanguagesNLP_present,Project Domain in Internship_Image Processing,College Project Domains_Image Processing,College Project Domains_Cloud computing,ElectiveCourseName_big data::A and higher
45
--------------------------------------------------------

time taken to get support of each combination :  0.27126598358154297
time taken to get valid combinations :  0.0
<class 'list'>
ProgLanguagesNLP_present,Project Domain in Internship_Image Processing,College Project Domains_Image Processing,College Project Domains_Cloud computing,College Project Domains_Data Structures,ElectiveCourseName_big data::A and higher
32
--------------------------------------------------------------------------------------------------
College Project Domains_Database Management System,ProgLanguagesNLP_present,Project Domain in Internship_Image Processing,College Project Domains_Image Processing,College Project Domains_Cloud computing,College Project Domains_Data Structures
32
--------------------------------------------------------------------------------------------------
College Project Domains_Operating System,College Project Domains_Database Management System,ProgLanguagesNLP_present,Project Domain in Internship_Image Processing,College Project Domains_Imag

In [680]:
comp= pd.read_csv(r"C:\Users\diyas\Downloads\CompanyDomains.csv") 
comp=comp.rename({'CompanyName_New':'Company Name'},axis=1)
a1=a.merge(comp,how="left",on="Company Name")

In [695]:
#GROUP RULES

def topRules(threshold, name):
    a_file = open(name)
    file_contents = a_file.read()
    rules = file_contents.splitlines()
    a_file.close()
    for i in range(len(rules)):
        temp=rules[i].split("[")   
        temp[0]=temp[0].strip("(").lstrip("'").rstrip("'").split(",")
        temp[0]=temp[0][0:-1]
        temp[0][-1]=temp[0][-1].rstrip("'")
        temp[1]=temp[1].replace(", ","")
        temp[1]=int(temp[1])
        temp[2]=temp[2].replace("]])","").strip("[")
        temp[2]=list(map(int, temp[2].split(',')))
        rules[i]=temp
    max_sup=rules[0][1]
    min_sup=rules[len(rules)-1][1]
    index_thresh=threshold*len(rules)
    index_thresh=int(index_thresh)
    thresh_sup=rules[index_thresh][1]
    thresh_rules=[]
    for i in rules:
        if(i[1]>=thresh_sup):
            thresh_rules.append(i)
    return thresh_rules

def rulesSep(rulesGroup, groups):
    groupDict={}
    for i in groups:
        for j in rulesGroup:
            if(i in j[0]):
                if(i not in groupDict):
                    groupDict[i]=[j]
                else:
                    groupDict[i].append(j)
    
    return groupDict
                
processedRules=topRules(0.6,"5_0.1_10_ElectiveCourseName_RulesElectiveCourse.txt")  
groups=['OffCampusLearning_present','NonAcademicSkills_present','ProgLanguagesNLP_present','SoftwareTools_present']
groupDict=rulesSep(processedRules,groups)

In [696]:
#for prog lang and software tools
# find the unique values in the Prog lang NLP and software tools 
#in the index of rulesGroup, if a given value of prog lang and software tools present then increase count of value
#find top n % of unique values in the column  

def drillDown(columns, df,groupDict):
    data=df.copy()
    for column in columns:
        col=column.replace("_present","")
        data[col]=data[col].str.strip('##').str.split('##')
        '''unique=[]
        for v,k in data[column].iteritems():
            for i in k:
                if i not in unique:
                    unique.append(i)'''
    items={}
    for group in groupDict:
        rules=groupDict[group]
        group_temp=group.replace("_present","")
        for rule in rules:
            temp_df=data.iloc[rule[2]]
            item_count={}
            for v,k in temp_df[group_temp].iteritems():
                for i in k:
                    if(i==""):
                        continue
                    if i in item_count:
                        item_count[i]=item_count[i]+1
                    else:
                        item_count[i]=1
            print(items, rule[0])
            s=str(rule[0])
            if str(rule[0]) not in items:
                items[s]={}
                items[s][group_temp]=item_count
            else:
                items[s][group_temp]=item_count
    return items
        

itemGroupCount=drillDown(groups, a1,groupDict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [697]:
itemGroupCountTop=itemGroupCount
for i in itemGroupCount.keys():
    for j in itemGroupCount[i].keys():
        newA = dict(sorted(itemGroupCount[i][j].items(), key=operator.itemgetter(1), reverse=True)[:5])
        itemGroupCountTop[i][j]=newA

In [698]:
#Company domain 
comp_domains=['Data Structures', 'Algorithms', 'Cloud computing',
       'Operating Systems', 'Web Development', 'Image Processing',
       'Database Management Systems', 'Machine Learning', 'Computer Network']
rule_dom_ct={}
for rule in processedRules:
    temp_df=a1.iloc[rule[2]]
    #print(temp_df.columns)
    for domain in comp_domains:
        item_count=0
        l=[]
        for v,k in temp_df.iterrows():
            #print(k[domain])
            if(k[domain]==1):
                item_count=item_count+1
                l.append(v)
        s=str(rule[0])
        if str(rule[0]) not in rule_dom_ct:
            rule_dom_ct[s]={}
            rule_dom_ct[s][domain]=[item_count,l]
        else:
            rule_dom_ct[s][domain]=[item_count,l]
        
    

In [699]:
rule_dom_sorted={}
for i in rule_dom_ct.keys():
    newA=sorted(rule_dom_ct[i].items(), key=lambda e: e[1][0], reverse=True)[:6]
    #print(newA)
    rule_dom_sorted[i]=newA

In [700]:
domain={}
for dom in comp_domains:
    for rule in rule_dom_sorted:
        for i in rule_dom_sorted[rule]:
            if(dom==i[0]):
                l=[]
                temp=rule.lstrip("['").rstrip("']").replace("'","").split(", ")
                temp[-1]=temp[-1].rstrip("'")
                l.append(temp)
                l.append(i[1][0])
                l.append(i[1][1])
                if dom not in domain:
                    domain[dom]=[]
                    domain[dom].append(l)
                else:
                    domain[dom].append(l)
                

In [701]:
domain['Computer Network']

[[['NonAcademicSkills_present',
   'College Project Domains_Database Management System',
   'Project Domain in Internship_Image Processing',
   'College Project Domains_Image Processing',
   'College Project Domains_Cloud computing'],
  9,
  [4, 16, 19, 66, 68, 71, 78, 92, 93]],
 [['NonAcademicSkills_present',
   'Project Domain in Internship_Image Processing',
   'College Project Domains_Image Processing',
   'College Project Domains_Cloud computing',
   'ElectiveCourseName_big data::A and higher'],
  11,
  [16, 19, 66, 68, 71, 80, 92, 181, 182, 252, 260]],
 [['ProgLanguagesNLP_present',
   'Project Domain in Internship_Image Processing',
   'College Project Domains_Image Processing',
   'College Project Domains_Cloud computing',
   'ElectiveCourseName_big data::A and higher'],
  8,
  [181, 182, 205, 206, 234, 237, 260, 284]],
 [['College Project Domains_Database Management System',
   'Project Domain in Internship_Image Processing',
   'College Project Domains_Image Processing',
   '

In [702]:
drillDownDom={}
for dom in domain:
    groupDict_dom=rulesSep(domain[dom],groups)
    itemGroupCount_dom=drillDown(groups, a1,groupDict_dom)
    itemGroupCountTop_dom=itemGroupCount_dom
    #print(itemGroupCount_dom)
    for i in itemGroupCount_dom.keys():
        for j in itemGroupCount_dom[i].keys():
            newA =sorted(itemGroupCount_dom[i][j].items(), key=operator.itemgetter(1), reverse=True)[:5]
            itemGroupCountTop_dom[i][j]=newA
    drillDownDom[dom]=itemGroupCountTop_dom
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{"['NonAcademicSkills_present', 'Project Domain in Internship_Image Processing', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing', 'OffCampusLearning_present']": {'OffCampusLearning': {'google': 1, 'german': 1, 'iisc': 1, 'nitk': 1, 'Robotics': 1, 'coursera': 4, 'deeplearning': 3, 'mooc': 1, 'stanford': 1, 'machinelearning': 1, 'datascience': 1, 'ieee': 1}}, "['NonAcademicSkills_present', 'Project Domain in Internship_Image Processing', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing', 'ElectiveCourseName_big data::A and higher']": {'NonAcademicSkills': {'ProblemSolving': 5, 'Inquisitive': 2, 'MentorforCAPS-PSRLLabatPESUniversitySports': 1, 'Instruments': 1, 'kannada': 4, 'hindi': 6, 'english': 6, 'telugu': 1, 'Leadership': 1, 'Communication': 1, 'TeamWork': 2, 'Motivated': 2, 'Timemanagement': 1, 'Organiserofcollegefest-Aatmatrisha,ClasscoordinatorinPrakalpaSciencefestTheatre': 1, 'SocialService': 1, 'Clubs': 2,

In [703]:
with open('Apriori Results Company Domain(Drill Down).json', 'w') as fp:
    fp.write((re.sub(r'",\s+', '", ', json.dumps(drillDownDom, indent=3))))

In [690]:
with open('Apriori Results Overall(Drill Down) FULL.json', 'w') as fp:
    fp.write((re.sub(r'",\s+', '", ', json.dumps(itemGroupCountTop, indent=3))))

In [709]:
with open('Apriori Results Company Domains.json', 'w') as fp:
    fp.write(json.dumps(rule_dom_sorted))

In [715]:
p={}
for i in rule_dom_sorted:
    print(i)
    for j in rule_dom_sorted[i]:
        print(j[0])
    print("------------------------------------------------------------------------")

['NonAcademicSkills_present', 'College Project Domains_Database Management System', 'Project Domain in Internship_Image Processing', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing']
Data Structures
Algorithms
Operating Systems
Web Development
Computer Network
Cloud computing
------------------------------------------------------------------------
['NonAcademicSkills_present', 'Project Domain in Internship_Image Processing', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing', 'ElectiveCourseName_big data::A and higher']
Data Structures
Algorithms
Operating Systems
Web Development
Computer Network
Database Management Systems
------------------------------------------------------------------------
['ProgLanguagesNLP_present', 'Project Domain in Internship_Image Processing', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing', 'ElectiveCourseName_big data::A and higher']
Data Structur

In [706]:
rule_dom_sorted

{"['College Project Domains_Database Management System', 'College Project Domains_Image Processing', 'College Project Domains_Cloud computing', 'College Project Domains_Data Structures', 'ElectiveCourseName_big data::A and higher']": [('Data Structures',
   [21,
    [52,
     58,
     66,
     69,
     106,
     109,
     110,
     126,
     187,
     200,
     203,
     205,
     206,
     211,
     214,
     215,
     229,
     230,
     255,
     286,
     287]]),
  ('Algorithms',
   [15,
    [58, 66, 69, 106, 126, 187, 203, 205, 206, 211, 229, 230, 255, 286, 287]]),
  ('Operating Systems',
   [12, [52, 106, 109, 110, 158, 159, 200, 203, 205, 206, 214, 215]]),
  ('Web Development',
   [11, [52, 109, 110, 200, 214, 215, 229, 230, 255, 265, 286]]),
  ('Database Management Systems', [7, [126, 158, 159, 229, 230, 255, 265]]),
  ('Cloud computing', [6, [106, 158, 159, 265, 286, 287]])],
 "['College Project Domains_Database Management System', 'ProgLanguagesNLP_present', 'College Project 